# CH 5 Basic Feature Engineering
---
### 1. What is Feature Engineering (FE)?

- Feature Engineering WRT TS(A+F) is reconstructing a time series problem to a supervised learning problem, specifically describing the `input features (X)` and `outputs (y)` that we would like to predict.
- **Goal:** To provide strong and ideally simple relationships between new input features and the output feature for the supervised learning algorithm to model
- Dataset: Minimum Daily Temperatures over 10 years (1981-1990) in the city Melbourne, Australia
---

### 2. Why is FE important?

- Time series problems have no concept of inputs and outputs so in order to make predictions, we must re-frame the problem to a model one that can make predictions with known inputs

---

### 3. What are some applications of FE? What other concepts can I connect to FE? Use FE when...

1. **Date Time Features:** these are components of the time step itself for each observation
    - Input features for each observation: (1) month -- int and (2) day -- int 
2. **Lag Features:** these are values at prior time steps
    - Input features for each observation: (1) t -- int and (2) t + 1
3. **Window Features:** these are a summary of values over a fixed window of prior time steps

---

### 4. What is the evolution/history of FE?

---

### 5. Can I predict the future use of FE? How can this current usage of FE improve?

---

### 6. What don't I understand about FE? Why is this? What's the root of this misunderstanding?

---

---

## Imports + Load Data

In [2]:
import pandas as pd
import sys  
sys.path.insert(0, "/Users/brinkley97/Documents/development/")
import my_created_functions

In [3]:
path_to_files = "book-intro_to_tsf_with_python/datasets/"
name_of_file = "daily-min-temperatures.csv"
dataset = my_created_functions.load_csv_data(path_to_files, name_of_file)
# dataset = load_csv_data(path_to_files, name_of_file)

In [4]:
# dataset

In [5]:
dataset_copy = dataset.reset_index()
dataset_copy

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8
...,...,...
3645,1990-12-27,14.0
3646,1990-12-28,13.6
3647,1990-12-29,13.5
3648,1990-12-30,15.7


## Transform Time Series Problem to Supervised Learning Problem Approach 1: Day Time Features
- Predict the daily minimum temperature `(y)`
- Input features `(X)` for each observation: `(1) month -- int` and `(2) day -- int`
- **Note**: This is NOT sophisticated and will likely result in a poor model. Would need additional engineered features (such as daylight savings or not, season of the year) for a better model.

In [57]:
# 1. convert the date datatype from an object to pandas datetime datatype
# dataset_copy["Date"]
dataset_copy["Date"] = pd.to_datetime(dataset_copy.Date)
dataset_copy["Date"]

0      1981-01-01
1      1981-01-02
2      1981-01-03
3      1981-01-04
4      1981-01-05
          ...    
3645   1990-12-27
3646   1990-12-28
3647   1990-12-29
3648   1990-12-30
3649   1990-12-31
Name: Date, Length: 3650, dtype: datetime64[ns]

In [66]:
# 2. create date-time features
date_time_features = pd.DataFrame()
date_time_features['month'] = dataset_copy['Date'].dt.month
date_time_features['day'] = dataset_copy['Date'].dt.day
date_time_features['temperature'] = dataset_copy['Temp']

In [67]:
date_time_features

,month,day,temperature
0,1,1,20.7
1,1,2,17.9
2,1,3,18.8
3,1,4,14.6
4,1,5,15.8
...,...,...,...
3645,12,27,14.0
3646,12,28,13.6
3647,12,29,13.5
3648,12,30,15.7


---

## Transform Time Series Problem to Supervised Learning Problem Approach 2 & 3: Lag Features & Window Features
- Predict the daily minimum temperature `(y)`, where `y = t + 1` (`y` is the next time step)
- Input features `(X)` for each observation: `(1) t -- int` and `(2) t + 1 -- int`
- **Note**: This is the classical way to transform our TS problem
- **Sliding Window Method:** as though we are sliding our focus along the time series for each observation with an interest in only what is within the window width. We can expand the window width and include more/less lagged features. Can be difficult to determine how large sliding window width should be so good starting point is to perform a sensitivity analysis and try a suite of different window widths to in turn create a suite of different views of your dataset and see which results in better performing models. We could use a non-linear approach: Today is 12/17/22. We want to predict tomorrow with 12/18/21, 12/18/21, 11/18/21, etc
   1. Window width = 1
   2. Window width = 3
   3. Window width = 2 with calculating the mean

### Sliding Window Method 1 

- Window width = 1
- Discard indices 0 - 1 as we don't have enough data
- We only have `X = t` to predict `y = t + 1`

In [7]:
dataset_copy_2 = dataset_copy
temp = dataset_copy_2["Temp"]
# temp

In [124]:
# t = temp.shift(2)
# t

In [122]:
# t_n = temp.shift()
# t_n

In [129]:
# 1 
window_width_1 = pd.concat([t, t_n], axis=1)
window_width_1.columns = ["t", "t + 1"]
window_width_1

,t,t + 1
0,NaN,NaN
1,NaN,20.7
2,20.7,17.9
3,17.9,18.8
4,18.8,14.6
...,...,...
3645,12.9,14.6
3646,14.6,14.0
3647,14.0,13.6
3648,13.6,13.5


### Sliding Window Method 2
- Window width = 3
- Discard indices 0 - 2 as we don't have enough data
- We have `X =` `t - 2`, `t - 1`, `t` to predict `y = t + 1`

In [14]:
window_width_3 = pd.concat([temp.shift(3), temp.shift(2), temp.shift(1), temp], axis=1)
window_width_3.columns = ["t - 2", "t - 1", "t", "t + 1"]
window_width_3

,t - 2,t - 1,t,t + 1
0,NaN,NaN,NaN,20.7
1,NaN,NaN,20.7,17.9
2,NaN,20.7,17.9,18.8
3,20.7,17.9,18.8,14.6
4,17.9,18.8,14.6,15.8
...,...,...,...,...
3645,10.0,12.9,14.6,14.0
3646,12.9,14.6,14.0,13.6
3647,14.6,14.0,13.6,13.5
3648,14.0,13.6,13.5,15.7


### Sliding Window Method 3
- Window width = 2 with calculating the mean
- Discard indices 0 - 1 as we don't have enough data
- We have `X =` `mean(t - 1, t)` to predict `y = t + 1`

In [22]:
shifted = temp.shift(1)
# creates a new data structure with the window of values at each time step; values of what we want to average
window = shifted.rolling(2)
# print(window)
means = window.mean()
window_width_2 = pd.concat([temp.shift(2), temp.shift(), means, temp], axis=1)
window_width_2.columns = ["t - 1","t", "mean(t - 1, t)", "t + 1"]
window_width_2

,t - 1,t,"mean(t - 1, t)",t + 1
0,NaN,NaN,NaN,20.7
1,NaN,20.7,NaN,17.9
2,20.7,17.9,19.30,18.8
3,17.9,18.8,18.35,14.6
4,18.8,14.6,16.70,15.8
...,...,...,...,...
3645,12.9,14.6,13.75,14.0
3646,14.6,14.0,14.30,13.6
3647,14.0,13.6,13.80,13.5
3648,13.6,13.5,13.55,15.7


In [23]:
window_width_2.rolling(2).mean()

,t - 1,t,"mean(t - 1, t)",t + 1
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,19.30
2,NaN,19.30,NaN,18.35
3,19.30,18.35,18.825,16.70
4,18.35,16.70,17.525,15.20
...,...,...,...,...
3645,11.45,13.75,12.600,14.30
3646,13.75,14.30,14.025,13.80
3647,14.30,13.80,14.050,13.55
3648,13.80,13.55,13.675,14.60
